# Langchain : basic

ที่มา: <a href="สอนการใช้งาน Langchain เบื้องต้น">https://medium.com/@leng.thod/%E0%B8%AA%E0%B8%AD%E0%B8%99%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%83%E0%B8%8A%E0%B9%89%E0%B8%87%E0%B8%B2%E0%B8%99-langchain-%E0%B9%80%E0%B8%9A%E0%B8%B7%E0%B9%89%E0%B8%AD%E0%B8%87%E0%B8%95%E0%B9%89%E0%B8%99-e451513a866b</a>

## Install required library

In [ ]:
# wikipedia ใช้ support plugin agents ตัวนึงของ langchain
# serpapi ใช้คู่กับ google-search-result
!pip install langchain
!pip install streamlit
!pip install openai==0.28
!pip install wikipedia 
!pip install google-search-results serpapi
!pip install langchain-community langchain-core
!pip install numexpr

## Api key

In [2]:
import os
import sys
sys.path.append('..')
from secret.secret_key import openapi_key, serapapi_key, second_openai_key
os.environ['OPENAI_API_KEY'] = second_openai_key

## What is Langchain

แอพตัวกลางช่วยเชื่อมต่อกับ LLM

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature = 0.6)#ต้องการให้สร้างสรรแค่ไหน - ค่าสูงสุด อยู่ที่ 1

In [ ]:
llm("please recommend me how to create resume for data analyst")

'\n\n1. Start with a clear and concise summary: Begin your resume with a brief summary that highlights your skills, experience, and achievements as a data analyst. This will give the hiring manager a quick overview of your qualifications.\n\n2. List your relevant skills: Make a list of your technical and analytical skills that are relevant to the data analyst role. This could include proficiency in programming languages, statistical analysis, data visualization tools, and database management.\n\n3. Include your work experience: List your previous job titles, company names, and employment dates in reverse chronological order. For each position, describe your key responsibilities and achievements using bullet points. Be sure to highlight any data analysis projects you have worked on and the results you have achieved.\n\n4. Showcase your education: If you have a degree in a relevant field such as mathematics, statistics, or computer science, be sure to include it on your resume. Also, men

- สามารถรับหลาย prompt เพื่อประมวลผลต่อเนื่องกันได้
- สามารถรับตัวแปรไปใน prompt และเปลี่ยนแปลงตัวแปรนั้นภายหลังได้

In [22]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
) # ส่งตัวแปร 'cuisine' เข้าไปใน prompt ผ่าน {} 

prompt_template_name.format(cuisine="Mexican") # แทนที่ cuisine ด้วย maxico

'I want to open a restaurant for Mexican food. Suggest a fancy name for this.'

In [ ]:
from langchain.chains import LLMChain
name_chain = LLMChain (llm=llm, prompt=prompt_template_name) #llm ที่มาจากการประกาศตัวแปรของเราด้านบน
name_chain.run("American") # แทนที่ cuisine ด้วย American

กรณี มีหลาย prompt ที่ต้องการให้ทำต่อเนื่องกัน ให้ใช้ SequentialChain

In [23]:
prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template = "I want to open restaurant for {cuisine} food. Suggest for fancy name for this."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key = 'restaurant_name') # เอา output จาก llm ใส่ตัวแปร restaurant_name

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma seperate list."
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'], #input_key ที่เข้ามา
    output_variables = ['restaurant_name', 'menu_items'] # output_key ที่ต้องการ
)

chain({'cuisine': 'Thai'})

# Agents

tools : Wikipedia, llm-math (คำนวณ)

In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [ ]:
import time
from langchain.callbacks.base import BaseCallbackHandler
# Avoid RPM of OpenAI model
class RateLimitCallbackHandler(BaseCallbackHandler):
    def on_llm_end(self, *args, **kwargs):
        time.sleep(20)  # sleep 20 seconds after each LLM response

# Attach handler to your LLM
from langchain.llms import OpenAI

llm_agent = OpenAI(temperature=0, callbacks=[RateLimitCallbackHandler()])


In [ ]:
tools =load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math เพราะเราจะทำการคิดเรรื่องคำนวณ

agent = initialize_agent(
    tools,
    llm_agent,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True #ให้เห็นผลแบบ step by step
                 
)

agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  ( EE-lon; born June 28, 1971) is a businessman. He is known for his leadership of Tesla, SpaceX, X (formerly Twitter), and the Department of Government Efficiency (DOGE). Musk has been considered the wealthiest person in the world since 2021; as of May 2025, Forbes estimates his net worth to be US$424.7 billion. 
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada. He received bachelor's degrees from the University of Pennsylvania in 1997 before moving to California, United States, to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, Musk also became an Ameri

'Elon Musk will be 52 years old in 2023.'

Tools: serpapi (google search), llm-math

In [5]:
os.environ['SERPAPI_API_KEY'] = serapapi_key

In [ ]:

import time
# llm = OpenAI(temperature=0)

tools =load_tools(["serpapi", "llm-math"], llm=llm) #llm-math เพราะเราจะทำการคิดเรรื่องคำนวณ
#
agent = initialize_agent(
    tools,
    llm_agent,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True #ให้แสดงการให้เห็นผลแบบ step by step
                 
)

# Define prompts (can be one or many)
prompts = [
    "What was the GDP of US in 2022 plus 5?",
]
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"
Observation: ['In the long-term, the United States GDP is projected to trend around 28643.00 USD Billion in 2026 and 29187.00 USD Billion in 2027, according to our econometric ...', 'Release Table for 2027, Release Tables: Summary of Economic Projections. FRED: Download, graph, and track economic data.', 'The initial read of real GDP growth for 2025Q1 indicated a contraction of 0.3 percent at an annual rate. Chart 1 shows the contributions of ...', "As of 2025, it has the world's seventh highest nominal GDP per capita and ninth highest GDP per capita by PPP. The U.S. accounts for 27% of the global economy ...", 'FOMC Summary of Economic Projections for the Growth Rate of Real Gross Domestic Product, Central Ten

'The GDP of the US in 2027 is projected to be around 29187.00 USD Billion with a growth rate of 2.3%.'

## Langchain: Memory management

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = OpenAI(temperature= 0.7))
print(convo.prompt.template)

In [29]:
convo.run("Who won the first cricket world cup")

' The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. Sir Clive Lloyd was the captain of the West Indies team and he scored 102 runs in the final. This was the first ever cricket world cup and it was held in England.'

In [30]:
convo.run("what is 5+5?")

' 5+5 is 10.'

In [31]:
convo.run("who was the captain of the winning team?")

' The captain of the winning team was Sir Clive Lloyd of the West Indies.'

In [ ]:
convo.memory # ณ ตอนนี้จะมี history ของตัวก่อนหน้ามาแล้ว

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Who won the first cricket world cup', additional_kwargs={}, response_metadata={}), AIMessage(content=' The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. Sir Clive Lloyd was the captain of the West Indies team and he scored 102 runs in the final. This was the first ever cricket world cup and it was held in England.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is 5+5?', additional_kwargs={}, response_metadata={}), AIMessage(content=' 5+5 is 10.', additional_kwargs={}, response_metadata={}), HumanMessage(content='who was the captain of the winning team?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The captain of the winning team was Sir Clive Lloyd of the West Indies.', additional_kwargs={}, response_metadata={})]))

In [ ]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup
AI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. Sir Clive Lloyd was the captain of the West Indies team and he scored 102 runs in the final. This was the first ever cricket world cup and it was held in England.
Human: what is 5+5?
AI:  5+5 is 10.
Human: who was the captain of the winning team?
AI:  The captain of the winning team was Sir Clive Lloyd of the West Indies.
